# TTS MODEL SETUP
* arabic:https://www.kaggle.com/code/mobassir/bn-nllb200-ar-tts#Arabic-Tacotron2-TTS : cell- 30 -->
* bangla:https://www.kaggle.com/code/mobassir/silero-bn-tts: cell 1-->


# Arabic TTS

In [1]:
!git clone https://github.com/nipponjo/tts-arabic-tacotron2
!pip install gdown

Cloning into 'tts-arabic-tacotron2'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 54 (delta 1), reused 54 (delta 1), pack-reused 0
Unpacking objects: 100% (54/54), 261.97 KiB | 1.38 MiB/s, done.
  Installing build dependencies ... - \ | / - \ done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
  Created wheel for gdown: filename=gdown-4.5.1-py3-none-any.whl size=14933 sha256=18c317797aaab8f18478924436563fc32f2fb4f974ca160dfff093b628acb405
  Stored in directory: /root/.cache/pip/wheels/3d/ec/b0/a96d1d126183f98570a785e6bf8789fca559853a9260e928e1
Successfully built gdown


In [2]:
import sys
sys.path.append('./tts-arabic-tacotron2')
import gdown
from model.networks import Tacotron2Wave
from IPython.display import Audio
import soundfile as sf
import shutil

In [3]:
url = "https://drive.google.com/drive/folders/196xZbqqxzsBQdKr1UKdh_wL_9qtJkt6u?usp=sharing"
gdown.download_folder(url=url, quiet=False, use_cookies=False)  

Retrieving folder list


Retrieving folder 1uHomQ-7dxwTc8DagzSjIIdCHiSEncROE UNIVERSAL_V1
Processing file 1_uRBHf9VmK6CGoB9c0Jb6nE8Gah7X6WI config.json
Processing file 1ejejcJ2l9AL0AnP79Ag-XK3HZzBYW64h do_02500000
Processing file 1oUGwFAAsRnM3OfwSWJVFJKyBmBZHc4fp g_02500000
Processing file 1CMvxg7zP1xoJbCLaJTbBCf1YU1SHl4uT tacotron2_ar.pth
Building directory structure completed


Retrieving folder list completed
Building directory structure
Downloading...
From: https://drive.google.com/uc?id=1_uRBHf9VmK6CGoB9c0Jb6nE8Gah7X6WI
To: /kaggle/working/arabic_tts/UNIVERSAL_V1/config.json
100%|██████████| 799/799 [00:00<00:00, 741kB/s]
Downloading...
From: https://drive.google.com/uc?id=1ejejcJ2l9AL0AnP79Ag-XK3HZzBYW64h
To: /kaggle/working/arabic_tts/UNIVERSAL_V1/do_02500000
100%|██████████| 960M/960M [00:05<00:00, 175MB/s]
Downloading...
From: https://drive.google.com/uc?id=1oUGwFAAsRnM3OfwSWJVFJKyBmBZHc4fp
To: /kaggle/working/arabic_tts/UNIVERSAL_V1/g_02500000
100%|██████████| 55.8M/55.8M [00:00<00:00, 160MB/s]
Downloading...
From: https://drive.google.com/uc?id=1CMvxg7zP1xoJbCLaJTbBCf1YU1SHl4uT
To: /kaggle/working/arabic_tts/tacotron2_ar.pth
100%|██████████| 338M/338M [00:01<00:00, 202MB/s]
Download completed


['/kaggle/working/arabic_tts/UNIVERSAL_V1/config.json',
 '/kaggle/working/arabic_tts/UNIVERSAL_V1/do_02500000',
 '/kaggle/working/arabic_tts/UNIVERSAL_V1/g_02500000',
 '/kaggle/working/arabic_tts/tacotron2_ar.pth']

In [4]:
shutil.copy('/kaggle/working/arabic_tts/UNIVERSAL_V1/config.json', './tts-arabic-tacotron2/pretrained/hifigan-universal-v1')

shutil.copy('/kaggle/working/arabic_tts/UNIVERSAL_V1/do_02500000', './tts-arabic-tacotron2/pretrained/hifigan-universal-v1')

shutil.copy('/kaggle/working/arabic_tts/UNIVERSAL_V1/g_02500000', './tts-arabic-tacotron2/pretrained/hifigan-universal-v1')

shutil.copy('/kaggle/working/arabic_tts/tacotron2_ar.pth', './tts-arabic-tacotron2/pretrained')

model_sd_path = './tts-arabic-tacotron2/pretrained/tacotron2_ar.pth'
# vocoder
vocoder_state_path = './tts-arabic-tacotron2/pretrained/hifigan-universal-v1/g_02500000'

vocoder_config_path = './tts-arabic-tacotron2/pretrained/hifigan-universal-v1/config.json'

In [5]:
%%writefile /kaggle/working/tts-arabic-tacotron2/utils/__init__.py
import sys
import yaml
try:
    from yaml import CLoader as Loader
except ImportError:
    from yaml import Loader


class DictConfig(object):
    """Creates a Config object from a dict 
       such that object attributes correspond to dict keys.    
    """

    def __init__(self, config_dict):
        self.__dict__.update(config_dict)

    def __str__(self):
        return '\n'.join(f"{key}: {val}" for key, val in self.__dict__.items())

    def __repr__(self):
        return self.__str__()


def get_custom_config(fname):
    with open(fname, 'r') as stream:
        config_dict = yaml.load(stream, Loader)
    config = DictConfig(config_dict)
    return config


def get_basic_config():
    return get_custom_config('./tts-arabic-tacotron2/configs/basic.yaml')


def get_config(fname):
    config = get_basic_config()
    custom_config = get_custom_config(fname)

    config.__dict__.update(custom_config.__dict__)
    return config


def read_lines_from_file(path, encoding='utf-8'):
    lines = []
    with open(path, 'r', encoding=encoding) as f:
        for line in f:
            lines.append(line.strip())
    return lines


def progbar(iterable, length=30, symbol='='):
    """Wrapper generator function for an iterable. 
       Prints a progressbar when yielding an item. \\
       Args:
          iterable: an object supporting iteration
          length: length of the progressbar
    """
    n = len(iterable)
    for i, item in enumerate(iterable):
        steps = length*(i+1) // n
        sys.stdout.write('\r')
        sys.stdout.write(f"[{symbol*steps:{length}}] {(100/n*(i+1)):.1f}%")
        if i == (n-1):
            sys.stdout.write('\n')
        sys.stdout.flush()
        yield item


Overwriting /kaggle/working/tts-arabic-tacotron2/utils/__init__.py


In [6]:

ar_model = Tacotron2Wave(model_sd_path = model_sd_path,vocoder_sd = vocoder_state_path, vocoder_config = vocoder_config_path)
ar_model = ar_model.cuda()

# Bangla TTS

In [7]:
!pip install aksharamukha
!pip install -q torchaudio omegaconf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.2/232.2 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for jaconv: filename=jaconv-0.3-py3-none-any.whl size=15564 sha256=c91e94d9273eb57df3e2157a9d25e7bc85f84e42dacb59886ce3fbd5c13dee55
  Stored in directory: /root/.cache/pip/wheels/8f/4f/c2/a2a3b14d0e94f855f4aa8887bf0267bee9ecfb8e62a9ee2d92
Successfully built jaconv


In [8]:
import torch
import torchaudio
from IPython.display import Audio, display
from aksharamukha import transliterate
import random

In [9]:
# Loading model
bn_model,_= torch.hub.load(repo_or_dir='snakers4/silero-models',
                           model='silero_tts',
                           language='indic',
                           speaker='v3_indic')

Downloading: "https://github.com/snakers4/silero-models/archive/master.zip" to /root/.cache/torch/hub/master.zip


  0%|          | 0.00/54.5M [00:00<?, ?B/s]

# TTS BigTextToAudio 

In [10]:
!pip install bnnumerizer

  Preparing metadata (setup.py) ... - done
  Created wheel for bnnumerizer: filename=bnnumerizer-0.0.2-py3-none-any.whl size=5277 sha256=8515cd2acc7d2d131d4a266ba2286d23407e0082beb7b67353289230ea72a775
  Stored in directory: /root/.cache/pip/wheels/18/e3/0e/47d630e869c084c60562cff8c709b8926ce22db40229a0afba
Successfully built bnnumerizer


In [11]:
import re
import torchaudio.functional as F
from bnnumerizer import numerize
"""
TODO: 
    * debug and traceback options
    
"""

class BigTextToAudio(object):
    
    def __init__(self,
                 ar_model,
                 bn_model,
                 ar_sample_rate=22050,
                 bn_sample_rate=48000,
                 out_sample_rate=22050,
                 attribution_dict={"সাঃ":"সাল্লাল্লাহু আলাইহি ওয়া সাল্লাম",                   
                                  "আঃ":"আলাইহিস সালাম",
                                  "রাঃ":"রাদিআল্লাহু আনহু",
                                  "রহঃ":"রহমাতুল্লাহি আলাইহি",
                                  "রহিঃ":"রহিমাহুল্লাহ",
                                  "হাফিঃ":"হাফিযাহুল্লাহ",
                                  "দাঃবাঃ":"দামাত বারাকাতুহুম,দামাত বারাকাতুল্লাহ"},
                text_batch_size=250,
                resample_params={"lowpass_filter_width": 64,
                                "rolloff": 0.9475937167399596,
                                "resampling_method": "kaiser_window",
                                "beta": 14.769656459379492}
                ):
        '''
            Instantiates a Big Text to Audio conversion object for bangla and arabic
            args:
                ar_model : arabic tts model
                bn_model : bangla tts model
                ar_sample_rate : arabic audio sample rate [optional] default: 22050
                bn_sample_rate : bangla audio sample rate [optional] default: 48000
                out_sample_rate : audio sample rate [optional] default: 22050
                attribution_dict : a dict for attribution expansion [optional]
                text_batch_size : chunk of long text to be considered as batch [optional]
                resample_params : audio resampling parameters [optional]
            resources:
                # Main class: modified from https://github.com/snakers4/silero-models/pull/174
                # Audio converter:https://www.kaggle.com/code/shahruk10/inference-notebook-wav2vec2
        '''
        self.ar_model = ar_model
        self.bn_model = bn_model
        self.attribution_dict=attribution_dict
        self.batch_size=text_batch_size
        self.ar_sample_rate=ar_sample_rate
        self.bn_sample_rate=bn_sample_rate
        self.sample_rate=out_sample_rate  
        self.resample_params=resample_params
        
    # public
    def ar_tts(self,text):
        '''
            args: 
                text: arabic text (string)
            returns:
                audio as torch tensor
        '''
        return self.ar_model.tts(text)
    # public
    def bn_tts(self,text,speaker):
        '''
            args: 
                text   : bangla text (string)
                speaker: which voice to use ["bengali_female","bengali_male"]
            returns:
                audio as torch tensor
        '''
        roman = transliterate.process('Bengali', 'ISO',text)
        return self.bn_model.apply_tts(roman,speaker=speaker)  
    
    # public
    def expand_full_attribution(self,text):
        '''
            expands full attribution
        '''
        for k,v in self.attribution_dict.items():text=text.replace(k,v)
        return text
    
    # public
    def tag_text(self,text):
        '''
            * tags arabic text with <ar>text</ar>
            * tags bangla text with <bn>text</bn>
        '''
        # remove multiple spaces
        text=re.sub(' +', ' ',text)
        # create start and end
        text="start"+text+"end"
        # tag text
        parts=re.split(u'[\u0600-\u06FF]+', text)
        # remove non chars
        parts=[p for p in parts if p.strip()]
        # unique parts
        parts=set(parts)
        # tag the text
        for m in parts:
            if len(m.strip())>1:text=text.replace(m,f"</ar><SPLIT><bn>{m}</bn><SPLIT><ar>")
        # clean-tags
        text=text.replace("</ar><SPLIT><bn>start",'<bn>')
        text=text.replace("end</bn><SPLIT><ar>",'</bn>')
        return text

    # private
    def __batch_text(self,text):
        '''
            batches a given text based on batch size
        '''
        batch=[]
        if len(text)<self.batch_size:
            batch=[text]
        else:
            for idx in range(0,len(text),self.batch_size):
                batch.append(text[idx:min(idx + self.batch_size, len(text))])
        return batch

    
    def process_text(self,text):
        '''
            private function to process text
                * expand attributions
                * numerize text
                * tag sections of the text
                * sequentially list text blocks
                * batch blocks
        '''
        # expand attributes
        text=self.expand_full_attribution(text)
        # numerize text
        text=numerize(text)
        # tag sections
        text=self.tag_text(text)
        # text blocks
        blocks=text.split("<SPLIT>")
        blocks=[b for b in blocks if b.strip()]
        # create tuple of (lang,text)
        data=[]
        for block in blocks:
            lang=None
            if "<bn>" in block:
                block=block.replace("<bn>",'').replace("</bn>",'')
                lang="bn"
            elif "<ar>" in block:
                block=block.replace("<ar>",'').replace("</ar>",'')
                lang="ar"
            # create batches
            if lang:
                batch=self.__batch_text(block)
                for _text in batch:
                    if _text.strip():
                        data.append((lang,_text))
        return data
    
    def resample_audio(self,audio,sr):
        '''
            resample audio with sample rate
            args:
                audio : torch.tensor audio
                sr: audi sample rate
        '''
        if sr==self.sample_rate:
            return audio
        else:
            return F.resample(audio,sr,self.sample_rate,**self.resample_params)
        
    
    def get_audio(self,data,bn_speaker):
        '''
            creates audio from given data 
                * data=List[Tuples(lang,text)]
        '''
        audio_list = []
        for block in data:
            lang,text=block
            if lang=="bn":
                audio=self.bn_tts(text,bn_speaker)
                sr=self.bn_sample_rate
            else:
                audio=self.ar_tts(text)
                sr=self.ar_sample_rate
            
            if self.resample_audio_to_out_sample_rate:
                audio=self.resample_audio(audio,sr)
                
            audio_list.append(audio)
        audio = torch.cat([k for k in audio_list])
        return audio
    
    # call
    def __call__(self,text,bn_speaker="bengali_female",resample_audio_to_out_sample_rate=True):
        '''
            args: 
                text   : bangla text (string)
                bn_speaker: which voice to use ["bengali_female","bengali_male"]
                            * default:"bengali_female"
                resample_audio_to_out_sample_rate: for different sample rate in different models, resample the output audio 
                                                   in uniform sample rate 
                                                   * default:True
            returns:
                audio as numpy data
        '''
        self.resample_audio_to_out_sample_rate=resample_audio_to_out_sample_rate
        data=self.process_text(text)
        audio=self.get_audio(data,bn_speaker)
        return audio.detach().cpu().numpy()

# Unit Tests and uses

In [12]:
TTS=BigTextToAudio(ar_model=ar_model,
                   bn_model=bn_model)
text="আহমাদ ৫/৩৩,৩৭,২৭ বাংলা কথা  اَلسَّلامُ عَلَيكُم يَا صَدِيقِي"

### Default female

In [13]:
audio=TTS(text)
Audio(audio, rate=TTS.sample_rate)

### Without resampling

In [14]:
audio=TTS(text,resample_audio_to_out_sample_rate=False)
Audio(audio, rate=TTS.sample_rate)

### Male speaker with resampling

In [15]:
audio=TTS(text,bn_speaker="bengali_male")
Audio(audio, rate=TTS.sample_rate)

### text processing check

In [16]:
data=TTS.process_text(text)
data

[('bn', 'আহমাদ পাঁচ/তেত্রিশ,সাঁইত্রিশ,সাতাশ বাংলা কথা '),
 ('ar', 'اَلسَّلامُ عَلَيكُم يَا صَدِيقِي')]

### ARABIC FAILURE

In [17]:
# TTS.ar_tts("بِسْمِ اللَّهِ الَّذِي لَا يَضُرُّ مَعَ اسْمِهِ شَيْءٌ فِي الْأَرْضِ وَلَا فِي السَّمَاءِ وَهُوَ السَّمِيعُ الْعَلِيمُ")